[View in Colaboratory](https://colab.research.google.com/github/nkcr/NETARCH/blob/master/Implementing%20a%20CNN%20for%20TC/Run%20Tensorboard.ipynb)

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

In [2]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0)
total = a+b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


Using TensorBoard

In [2]:
import tensorflow as tf
tf.reset_default_graph()
#LOG_DIR = '/tmp/log'
LOG_DIR= '/content/runs/'
#LOG_DIR= '/content/runs/1526979864/checkpoints'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
! wget -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -n ngrok-stable-linux-amd64.zip

#writer = tf.summary.FileWriter('.')
#writer.add_graph(tf.get_default_graph())
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


--2018-05-28 07:22:53--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.225.24.230, 52.1.117.85, 34.239.63.98, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.225.24.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  8.54MB/s    in 0.6s    

2018-05-28 07:22:54 (8.54 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://e9b237d8.ngrok.io


In [0]:
writer = tf.summary.FileWriter(LOG_DIR)
writer.add_graph(tf.get_default_graph())

In [0]:
! cd runs && rm -rf *

In [38]:
! ps -ef | grep 6006
! kill -9 23162 23190
! ps -ef | grep 6006

root     23162     1  0 09:41 ?        00:00:02 /usr/bin/python2 /usr/local/bin/tensorboard --logdir /content/runs/1526979864/checkpoints --host 0.0.0.0 --port 6006
root     23190     1  0 09:41 ?        00:00:03 ./ngrok http 6006
root     23290   191 99 09:57 pts/0    00:00:01 /bin/sh -c  ps -ef | grep 6006
root     23292 23290  0 09:57 pts/0    00:00:00 grep 6006
root     23294   191 99 09:57 pts/0    00:00:01 /bin/sh -c  ps -ef | grep 6006
root     23296 23294  0 09:57 pts/0    00:00:00 grep 6006


In [6]:
! ls -l /content/runs/

total 4
drwxr-xr-x 4 root root 4096 May 24 09:31 1527154306


In [0]:
rm -rf runs/1527153935

In [1]:
FILE="GoogleNews-vectors-negative300.bin"
!wget -nc 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
!ls
get_ipython().system_raw(
    '[ -f "{}" ] && echo "skipping {}" || gunzip "{}.gz"'
    .format(FILE,FILE,FILE)
)
!ls

--2018-05-28 07:16:45--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.102.149
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.102.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

e300.bin.gz          74%[=============>      ]   1.15G  44.8MB/s    eta 15s    

GoogleNews-vectors- 100%[===================>]   1.53G  45.1MB/s    in 52s     

2018-05-28 07:17:38 (30.0 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]

datalab  GoogleNews-vectors-negative300.bin.gz
datalab  GoogleNews-vectors-negative300.bin


In [42]:
from google.colab import files
files.download('projector_metadata.tsv')

MessageError: ignored

Run a session

In [7]:
sess = tf.Session()
print(sess.run(total))
print(sess.run({'ab': (a,b), 'total': total}))

7.0
{'total': 7.0, 'ab': (3.0, 4.0)}


In [8]:
vec = tf.random_uniform(shape=[3])
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(out1))
print(sess.run(out2))
print(sess.run((out1, out2)))

[0.9237155  0.42957187 0.6805531 ]
[1.3432978 1.8276473 1.9982173]
[2.5280676 2.34945   2.8061225]
(array([1.084274 , 1.5919821, 1.880151 ], dtype=float32), array([2.084274 , 2.5919821, 2.880151 ], dtype=float32))


Placeholder are a promise to provide a value later

In [0]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [10]:
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1,2], y: [2,4]}))

7.5
[3. 6.]


Dataset

In [11]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]

slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

while True:
  try:
    print(sess.run(next_item))
  except tf.errors.OutOfRangeError:
    break
    
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)

sess.run(iterator.initializer)
while True:
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break

[0 1]
[2 3]
[4 5]
[6 7]
[-0.7685284 -2.0994887 -1.3657249]
[ 1.9649249 -0.5264158  0.449746 ]
[-0.45013475  0.2322913   1.2542223 ]
[ 0.6559432  2.124139  -1.4459771]
[-0.820264    0.49138325 -0.02158285]
[-0.41786754 -0.20539466 -0.1764493 ]
[1.7423459 0.9923604 1.0931443]
[ 0.14025863  0.29250455 -0.33688542]
[-0.08791608 -2.150556   -0.25590885]
[ 0.07763281 -1.0634928   0.07647065]


Speaking about layers, which embed operations and data

In [12]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(y, feed_dict= {x: [[1,2,3],[2,3,1]]}))

y = tf.layers.dense(x, units=1)


[[ 2.4900746 ]
 [-0.10331476]]


In [0]:
writer.add_graph(tf.get_default_graph())

Feature columns

In [0]:
# We must wrap categorical column with indicator_column
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
                      'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

In [40]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

(None, None)

In [41]:
print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


Training


In [0]:
x = tf.constant([[1], [2], [3], [4], [20]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3], [-19]], dtype=tf.float32)

In [0]:
linear_model = tf.layers.Dense(units=1)
y_pred = linear_model(x)

In [127]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[ -0.50970185]
 [ -1.0194037 ]
 [ -1.5291055 ]
 [ -2.0388074 ]
 [-10.1940365 ]]


In [128]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)
print(sess.run(loss))

15.79016


In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [130]:
for i in range(1000):
  _, loss_value = sess.run((train, loss))
  if i%100 == 0:
    print(loss_value)

15.79016
0.06007568
0.0058583817
0.0005712978
5.5709515e-05
5.4329516e-06
5.299848e-07
5.1702187e-08
5.028807e-09
4.950067e-10


In [131]:
print(sess.run(y_pred))
print(sess.run(linear_model(tf.constant([[0],[1],[2],[3],[4],[30]], dtype=tf.float32))))

[[-8.2254410e-06]
 [-1.0000076e+00]
 [-2.0000069e+00]
 [-3.0000062e+00]
 [-1.8999996e+01]]
[[ 9.9999112e-01]
 [-8.2254410e-06]
 [-1.0000076e+00]
 [-2.0000069e+00]
 [-3.0000062e+00]
 [-2.8999990e+01]]


# Tensors

In [0]:
# Rank 0

mammal = tf.Variable("Elephant", tf.string)
ignition = tf.Variable(342, tf.int16)
floating = tf.Variable(3.3, tf.float64)
its_complicated = tf.Variable(12.3 - 3.1j, tf.complex64)

In [0]:
# Rank 1
mystr = tf.Variable(["Hello"], tf.string
                   )
cool_numbers = tf.Variable([12.34, 234.3], tf.float16)

#Higher rank
mymat = tf.Variable([[1], [2], [1]], tf.int16)

In [99]:
print(tf.rank(mymat))

Tensor("Rank_3:0", shape=(), dtype=int32)


In [0]:
from sklearn.datasets import fetch_rcv1
rcv1 = fetch_rcv1()

In [28]:
print(list(zip(rcv1.data[:20], rcv1.target[:20]))[0][0].toarray())

[[0. 0. 0. ... 0. 0. 0.]]
